In [ ]:
# Install Ollama and pyngrok
!curl -fsSL https://ollama.com/install.sh | sh
!pip install pyngrok flask requests


>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
from pyngrok import ngrok, conf
import time, os

# 🔧 USER SETTINGS
NGROK_AUTHTOKEN = "2ygjBSFtr1u6JbaEGjN6rK0vxie_3VzUt6MbZ55URyvNwm6nS"   # @param {type:"string"}
NGROK_DOMAIN    = "hugely-chief-dingo.ngrok-free.app"   # @param {type:"string"}
API_KEY         = "sec"  # @param {type:"string"}
MODEL           = "qwen2.5:7b-instruct"

if not NGROK_AUTHTOKEN:
    raise ValueError("Enter your ngrok authtoken!")
if not NGROK_DOMAIN:
    raise ValueError("Enter your reserved NGROK_DOMAIN!")

# Configure ngrok region
conf.get_default().region = "eu"

# Login to ngrok
ngrok.set_auth_token(NGROK_AUTHTOKEN)

# Kill old processes
!pkill -f ollama || echo "ok"
!fuser -k 8000/tcp || echo "ok"

# Start Ollama daemon
get_ipython().system_raw("OLLAMA_HOST=0.0.0.0 ollama serve &")

print("🧠 Warming Ollama...")
time.sleep(5)

# Pull model
!ollama pull $MODEL


^C
8000/tcp:             2382
🧠 Warming Ollama...



In [ ]:
%%writefile api.py
from flask import Flask, request, jsonify
import requests as http

API_KEY = "sec"
MODEL   = "qwen2.5:7b-instruct"

app = Flask(__name__)

# ----------------------------------------
# 🔍 1) MODELS LIST ENDPOINT (OpenAI style)
# ----------------------------------------
@app.route("/v1/models", methods=["GET"])
def list_models():
    try:
        r = http.get("http://localhost:11434/api/tags")
        if r.status_code != 200:
            return jsonify({
                "object": "list",
                "data": [],
                "error": "could not load models from ollama"
            }), 500

        ollama_models = r.json().get("models", [])

        models_out = [
            {
                "id": m["name"],
                "object": "model",
                "owned_by": "ollama",
                "permission": []
            }
            for m in ollama_models
        ]

        return jsonify({
            "object": "list",
            "data": models_out
        })

    except Exception as e:
        return jsonify({"error": str(e)}), 500


# -----------------------------------------------------
# 💬 2) CHAT COMPLETIONS (OpenAI-compatible wrapper)
# -----------------------------------------------------
@app.route("/v1/chat/completions", methods=["POST"])
def completions():
    # --- API key validation ---
    auth = request.headers.get("Authorization", "").replace("Bearer ", "")
    if auth != API_KEY:
        return jsonify({"error": "unauthorized"}), 401

    data = request.json
    user_msg = data["messages"][0]["content"]

    # --- Send request to Ollama (non-streaming) ---
    r = http.post(
        "http://localhost:11434/api/chat",
        json={
            "model": MODEL,
            "messages": [{"role": "user", "content": user_msg}],
            "stream": False
        }
    )

    if r.status_code != 200:
        return jsonify({
            "error": "ollama_error",
            "details": r.text
        }), 500

    reply = r.json()["message"]["content"]

    # --- Return OpenAI-style structure ---
    return jsonify({
        "id": "chatcmpl-ollama",
        "object": "chat.completion",
        "model": MODEL,
        "choices": [
            {
                "index": 0,
                "message": {
                    "role": "assistant",
                    "content": reply
                },
                "finish_reason": "stop"
            }
        ]
    })


# -----------------------------------------------------
# 🚀 3) API Runner
# -----------------------------------------------------
if __name__ == "__main__":
    print("🚀 API running at http://0.0.0.0:8000")
    app.run(host="0.0.0.0", port=8000)


Overwriting api.py


In [ ]:
# Kill old server
!pkill -f api.py || echo "ok"
!fuser -k 8000/tcp || echo "ok"

# Start Flask server permanently
!nohup python3 api.py >/dev/null 2>&1 &

# Wait for startup
import time; time.sleep(3)

# Verify port
!lsof -i:8000


^C
ok
COMMAND  PID USER   FD   TYPE DEVICE SIZE/OFF NODE NAME
python3 4421 root    3u  IPv4 129447      0t0  TCP *:8000 (LISTEN)


In [ ]:
public_url = ngrok.connect(
    addr=8000,
    proto="http",
    hostname=NGROK_DOMAIN
)

print("=======================================")
print("🚀 API is LIVE at:", public_url)
print("🔑 API KEY:", API_KEY)
print("=======================================")


🚀 API is LIVE at: NgrokTunnel: "https://hugely-chief-dingo.ngrok-free.app" -> "http://localhost:8000"
🔑 API KEY: sec
